# EDA Project: Data Analysis for Marketing


## Introduction
A fast-food company came to us (Data Solutions company) to help them with opening their first branch in NY metropolitan area.  Their goal is to maximize their sales and reduce their operation cost. In order to achieve this target. We will need to know the best month of the year to open the branch based on the busyness and the amount of people in targeted stations. Additionally, we will try to locate the best location to open the branch.  Furthermore, we will analyze the given data to calculate the busiest months, days of the week and hours. This will help in planning the necessary offers by reducing them during rush days and hours.     

# 1.Data Gathering and Cleaning:
For this project, I will use a whole year of data becuase it is rquiered to caculate the busyiest month of the year. The the rent of the location and the opening of the branch is qoing to take place at the begning of this month. There are some nessary things that we need to take care of for the sake of effciancy and understanding:
- Dividing the data into 12 months. This will speed up the process.
    - It is requierd to use the latest data.  
- The busyness is calculated by the sum of ENTRIES and EXITS. 


First, we should importall the nessary libraries and keep them on the top. 

In [14]:
import pandas as pd

Second, any function will be in the following: 

In [15]:
# This function will take the numbers of the week to and pull them from the directory of this project.
# The function returns a concatneted DataFram of svereal txt files.  
def get_data(week_nums):
    loc = "turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_loc = loc.format(week_num)
        dfs.append(pd.read_csv(file_loc))
    return pd.concat(dfs)


# 2. The Busiest Station:


### 2.1 January:

In [35]:
january_weeks= [210102, 210109, 210116, 210123]
jan_df = get_data(january_weeks)
jan_df.head(2)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,03:00:00,REGULAR,7508509,2557569
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/26/2020,07:00:00,REGULAR,7508511,2557581


### 2.2 February:

In [33]:
february_weeks= [210206, 210213, 210220, 210227]
feb_df = get_data(february_weeks)
feb_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/30/2021,03:00:00,REGULAR,7524539,2564693
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,01/30/2021,07:00:00,REGULAR,7524543,2564703


### 2.3 March:

In [32]:
march_weeks= [210306, 210313, 210320, 210327]
mar_df = get_data(march_weeks)
mar_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,03:00:00,REGULAR,7537255,2570434
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/27/2021,07:00:00,REGULAR,7537260,2570445


### 2.4 April:

In [30]:
april_weeks= [210403, 210417, 210424]
apr_df = get_data(april_weeks)
apr_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/27/2021,00:00:00,REGULAR,7550517,2576642
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/27/2021,04:00:00,REGULAR,7550517,2576642


### 2.5 May:

In [28]:
may_weeks= [210501, 210508, 210515, 210522, 210529]
may_df = get_data(may_weeks)
may_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2021,00:00:00,REGULAR,7561288,2582898
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2021,04:00:00,REGULAR,7561295,2582903


### 2.6 June:

In [29]:
june_weeks= [210605, 210612, 210619, 210626]
jun_df = get_data(june_weeks)
jun_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/29/2021,00:00:00,REGULAR,7578734,2590325
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/29/2021,04:00:00,REGULAR,7578740,2590327


### 2.7 July:

In [25]:
july_weeks= [210703, 210710, 210717, 210724, 210731]
jul_df = get_data(july_weeks)
jul_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/26/2021,00:00:00,REGULAR,7592792,2595706
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/26/2021,04:00:00,REGULAR,7592804,2595713


### 2.8 August:

In [24]:
august_weeks= [210807, 210814, 210821, 210828]
aug_df = get_data(august_weeks)
aug_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/31/2021,00:00:00,REGULAR,7611181,2603110
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/31/2021,04:00:00,REGULAR,7611192,2603113


### 2.9 September:

In [23]:
sep_weeks= [210911, 210918, 210925]
sep_df = get_data(sep_weeks)
sep_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/04/2021,00:00:00,REGULAR,7629452,2610530
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/04/2021,04:00:00,REGULAR,7629460,2610532


### 2.10 October:

In [17]:
october_weeks= [201003, 201010, 201017, 201024]
oct_df = get_data(october_weeks)
oct_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/26/2020,00:00:00,REGULAR,7463495,2538694
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/26/2020,04:00:00,REGULAR,7463500,2538697


### 2.11 November: 

In [21]:
nov_weeks= [201107, 201114, 201121, 201128]
nov_df = get_data(nov_weeks)
nov_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,00:00:00,REGULAR,7481098,2545767
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/31/2020,04:00:00,REGULAR,7481103,2545767


### 2.12 December: 

In [22]:
dec_weeks= [201205, 201212, 201219, 201226]
dec_df = get_data(dec_weeks)
dec_df.head(2)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/28/2020,03:00:00,REGULAR,7495312,2551701
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/28/2020,07:00:00,REGULAR,7495318,2551711


# 3. The Busiest Month:


# 4. The Busiest Hours:

# 5. Findings and Conclusions:
